# LangGraph test

In [ ]:
!pip install langgraph dotenv langchain langchain-openai langchain-community openai faiss-cpu rank_bm25

# set env

In [62]:
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import os
from collections import defaultdict

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

faiss_store = (
    FAISS.load_local(
        "faiss_store", embeddings=embeddings, allow_dangerous_deserialization=True
    )
    if os.path.exists("faiss_store")
    else None
)

all_docs = list(faiss_store.docstore._dict.values())


all_texts = [doc.page_content for doc in faiss_store.docstore._dict.values()]

# FAISS + ensemble[bm25 / similarity]


In [78]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

common_docs = [doc for doc in all_docs if doc.metadata.get("main_topic") == "공통 구조 해석"]

# 나머지 카테고리별로 문서 그룹화
specific_docs_grouped = defaultdict(list)
for doc in all_docs:
    category = doc.metadata.get("main_topic")
    if category and category != "공통 구조 해석":
        specific_docs_grouped[category].append(doc)

# BM25 리트리버 캐시 생성: 각 카테고리에 '공통' 문서를 합쳐서 생성
bm25_retriever_cache = {}
for category, docs in specific_docs_grouped.items():
    # 해당 카테고리 문서와 공통 문서를 합칩니다.
    combined_docs = docs + common_docs
    bm25_retriever_cache[category] = BM25Retriever.from_documents(combined_docs)

# '공통 해석 사항' 자체에 대한 리트리버도 추가 (필요 시 사용)
if common_docs:
    bm25_retriever_cache["공통 구조 해석"] = BM25Retriever.from_documents(common_docs)

print(f"✅ BM25 캐시 생성 완료! 대상: {list(bm25_retriever_cache.keys())}")

def get_ensemble_retriever_by_category(
    faiss_store: FAISS,
    bm25_cache: dict,
    category: str,
    k: int = 3,
    weights: list[float] = [0.3, 0.7]
):
    """
    지정된 카테고리(+공통)에 맞는 리트리버를 동적으로 조합하여 반환합니다.
    """
    # 1. 캐시에서 미리 생성된 BM25 리트리버를 가져옵니다.
    # 이 리트리버는 이미 '공통' 문서를 포함하고 있습니다.
    sparse_bm25_retriever = bm25_cache.get(category)
    if not sparse_bm25_retriever:
        raise ValueError(f"'{category}' 카테고리에 대한 BM25 리트리버가 캐시에 없습니다.")
    sparse_bm25_retriever.k = k

    # 2. FAISS 리트리버는 'filter'와 '$in'을 사용하여 동적으로 생성합니다.
    # '공통'과 '지정된 카테고리'를 모두 포함하도록 필터링합니다.
    search_categories = ["공통 구조 해석", category]
    # 만약 카테고리가 '공통 해석 사항'이라면 중복을 피합니다.
    if category == "공통 구조 해석":
        search_categories = ["공통 구조 해석"]
    
    dense_similarity_retriever = faiss_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": k, 
            # 중요: 메타데이터 키를 'main_topic'으로 통일합니다.
            "filter": {"main_topic": {"$in": search_categories}}
        }
    )

    # 3. 두 리트리버를 앙상블로 묶습니다.
    return EnsembleRetriever(
        retrievers=[sparse_bm25_retriever, dense_similarity_retriever],
        weights=weights,
    )




K = 3
weights = [0.3, 0.7]

# Sparse
sparse_bm25_retriever = BM25Retriever.from_texts(texts=all_texts)
sparse_bm25_retriever.k = K  # k값을 통일하여 설정
# Dense
dense_similarity_retriever = faiss_store.as_retriever(
    search_type="similarity", search_kwargs={"k": K}
)

retriever = EnsembleRetriever(
    retrievers=[sparse_bm25_retriever, dense_similarity_retriever],
    weights=weights,
)

✅ BM25 캐시 생성 완료! 대상: ['집', '나무', '사람', '공통 구조 해석']


# state 정의

In [79]:
from typing import List, TypedDict

class GraphState(TypedDict):
    """
    Graph RAG 파이프라인의 상태

    Attributes:
        original_question (str): 사용자가 입력한 원본 질문 (그림 묘사)
        decomposed_questions (List[str]): 의미 단위로 분해된 질문 리스트
        retrieved_contexts (List[str]): 검색된 관련 문서(해석) 조각 리스트
        generation (str): LLM이 생성한 최종 해석
        relevance_check (str): 질문의 HTP 검사 관련성 여부 ("yes" or "no")
        hallucination_check (str): 생성된 답변의 환각 현상 유무 ("yes" or "no")
        category (str): 질문 범주(집, 나무, 사람)
    """
    original_question: str
    decomposed_questions: List[str]
    retrieved_contexts: List[str]
    generation: str
    relevance_check: str
    hallucination_check: str
    category: str

# 실행 시간 체크

In [80]:
import time
import functools

execution_times = {}

def timing_decorator(func):
    """
    각 노드 실행 시간 측정
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Execution time for {func.__name__}: {elapsed_time:.2f} seconds")
        execution_times[func.__name__] = elapsed_time
        return result
    return wrapper

# 노드 정의

In [81]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableConfig

# 2-1. Relevance Check Node (질문 관련성 검사)
@timing_decorator
def relevance_check_node(state: GraphState):
    """
    입력된 질문이 HTP 심리검사 해석과 관련이 있는지 확인합니다.
    """
    print("--- 1. 질문 관련성 검사 시작 ---")
    question = state["original_question"]
    
    prompt = ChatPromptTemplate.from_template(
        """당신은 심리검사 전문가입니다. 주어진 질문이 'HTP(집-나무-사람) 그림 심리검사' 해석과 관련된 내용인지 판단해주세요.
        HTP 검사는 집, 나무, 사람 그림의 특징(예: 지붕, 문, 창문, 나무 기둥, 가지, 사람의 눈, 코, 입 등)을 분석하는 것입니다.
        질문은 HTP 그림에 대한 관찰 묘사로, 그림의 요소나 특징에 대한 내용입니다.
        이에 해당하면 'yes', 전혀 관련 없는 내용(예: 오늘 날씨, 스포츠 경기 결과 등)이면 'no'로만 대답해주세요.

        질문: {question}
        판단 (yes/no):"""
    )
    
    chain = prompt | llm | StrOutputParser()
    relevance = chain.invoke({"question": question})
    
    print(f"질문 관련성: {relevance}")
    state["relevance_check"] = relevance.lower()
    if state["relevance_check"] == "no":
        # print("질문이 HTP 검사와 관련이 없습니다. 프로세스를 종료합니다.")
        state["generation"] = "관찰 결과를 다시 입력해주세요."
        print(state)
    return state

# 2-2. Decompose Node (질문 분해)
@timing_decorator
def decompose_query_node(state: GraphState):
    """
    입력된 질문을 의미 단위의 여러 하위 질문으로 분해합니다.
    """
    print("--- 2. 질문 분해 시작 ---")
    question = state["original_question"]
    
    prompt = ChatPromptTemplate.from_template(
        """당신은 HTP 그림 심리검사 문장 분석 전문가입니다. 상담사가 그림을 보고 관찰한 내용을 나열한 문장이 주어집니다. 
        이 문장을 그림의 각 요소(예: 문, 창문, 지붕, 길 등)에 대한 독립적인 해석이 가능한 단위로 분해하여 JSON 리스트 형태로 반환해주세요.

        예시:
        입력: "집에 창문은 2개 존재하고 크기는 적절함. 문은 집의 크기에 비해 작으며 문과 바깥이 길로 이어져 있지 않음."
        출력: {{"queries": ["집 창문의 개수는 2개이고 크기는 적절하다.", "집 문의 크기가 집 전체에 비해 작다.", "집과 외부를 잇는 길이 그려져 있지 않다."]}}

        입력: {question}
        출력:"""
    )
    
    chain = prompt | llm | JsonOutputParser()
    decomposed = chain.invoke({"question": question})
    
    decomposed_questions = decomposed.get("queries", [])
    print(f"분해된 질문: {decomposed_questions}")
    state["decomposed_questions"] = decomposed_questions
    return state

# 2-3. Retrieve Node (정보 검색)
@timing_decorator
def meta_retrieve_node(state: GraphState, config: RunnableConfig):
    """
    분해된 각 질문에 대해, state의 category에 맞춰 동적으로 Ensemble Retriever를
    생성하고 관련 문서를 검색합니다.
    """
    print(f"--- 3. 정보 검색 시작 (카테고리: {state['category']}) ---")
    
    # FastAPI의 app.state에 저장해 둔 구성요소를 config를 통해 가져옵니다.
    faiss_store = config["configurable"]["faiss_store"]
    bm25_cache = config["configurable"]["bm25_retriever_cache"]
    
    # 현재 요청의 카테고리에 맞는 앙상블 리트리버를 즉시 생성합니다.
    try:
        retriever = get_ensemble_retriever_by_category(
            faiss_store=faiss_store,
            bm25_cache=bm25_cache,
            category=state["category"]
        )
    except ValueError as e:
        # 특정 카테고리가 없는 경우에 대한 예외 처리
        print(e)
        state["retrieved_contexts"] = []
        # 또는 에러 메시지를 generation에 담아 사용자에게 전달할 수도 있습니다.
        state["generation"] = "죄송합니다. 요청하신 카테고리의 정보를 찾을 수 없습니다."
        return state

    decomposed_questions = state["decomposed_questions"]
    all_retrieved_docs = []
    for query in decomposed_questions:
        print(f"  - 검색 쿼리: '{query}'")
        retrieved_docs = retriever.invoke(query)
        doc_texts = [doc.page_content for doc in retrieved_docs]
        all_retrieved_docs.extend(doc_texts)
    
    unique_contexts = list(set(all_retrieved_docs))
    print(f"검색된 해석 Context 수: {len(unique_contexts)}")
    state["retrieved_contexts"] = unique_contexts
    return state

@timing_decorator
def retrieve_node(state: GraphState):
    """
    분해된 각 질문에 대해 Ensemble Retriever를 사용하여 관련 문서를 검색합니다.
    """
    print("--- 3. 정보 검색 시작 ---")
    decomposed_questions = state["decomposed_questions"]
    all_retrieved_docs = []

    for query in decomposed_questions:
        print(f"  - 검색 쿼리: '{query}'")
        # 여기서 사용자가 제공한 retriever를 사용합니다.
        retrieved_docs = retriever.invoke(query)
        
        # 검색 결과의 내용을 문자열로 변환하여 추가
        doc_texts = [doc.page_content for doc in retrieved_docs]
        all_retrieved_docs.extend(doc_texts)
    
    # 중복 제거
    unique_contexts = list(set(all_retrieved_docs))
    print(f"검색된 해석 Context 수: {len(unique_contexts)}")
    state["retrieved_contexts"] = unique_contexts
    return state

# 2-4. Generate Node (답변 생성)
@timing_decorator
def generate_node(state: GraphState):
    """
    검색된 Context를 바탕으로 최종 해석 답변을 생성합니다.
    """
    print("--- 4. 답변 생성 시작 ---")
    question = state["original_question"]
    contexts = "\n\n".join(state["retrieved_contexts"])
    
    prompt = ChatPromptTemplate.from_template(
        """당신은 HTP 그림 심리검사 결과 해석 전문가입니다.
        상담사가 관찰한 '그림 특징'과 그에 대한 '해석 참고자료'가 주어집니다. 
        두 정보를 종합하여, 내담자의 심리상태에 대한 최종 해석 보고서를 작성해주세요.
        전문적이고 이해하기 쉬운 말투로 설명하고, 각 특징과 해석을 논리적으로 연결하여 설명해주세요.
        정보는 반드시 주어진 '해석 참고자료'에 근거해야 하며, 환각(hallucination)이 없어야 합니다.
        답변은 질문에 대한 관찰 내용과 관련된 것만 작성해야 합니다.
        각 특징에 대한 해석 내용을 분석하고 참고 자료에서 관련 내용을 찾아 종합적으로 해석하세요.
        답변은 너무 극단적이지 않고 충분히 납득할 수 있는 수준으로 작성해야 합니다.
        참고자료가 너무 적거나 관련성이 낮다면, 답변은 간단하게 작성해주세요.

        ## 상담사의 그림 특징 관찰 내용:
        {question}

        ## 해석 참고자료:
        {context}

        ## 최종 해석 보고서:
        """
    )
    
    chain = prompt | llm | StrOutputParser()
    generation = chain.invoke({"question": question, "context": contexts})
    
    print("생성된 답변 일부:", generation[:200] + "...")
    state["generation"] = generation
    return state

# 2-5. Hallucination Check Node (환각 검사)
@timing_decorator
def hallucination_check_node(state: GraphState):
    """
    생성된 답변에 환각(hallucination)이 있는지 검사합니다.
    """
    print("--- 5. 환각 검사 시작 ---")
    contexts = state["retrieved_contexts"]
    generation = state["generation"]
    
    prompt = ChatPromptTemplate.from_template(
        """당신은 AI 답변 검증 전문가입니다. 주어진 '참고 자료'를 바탕으로 '생성된 답변'이 만들어졌는지 확인해야 합니다.
        '생성된 답변'의 모든 내용이 '참고 자료'에 근거하고 있다면 'yes'를, '참고 자료'에 없는 내용이 포함되어 있다면 'no'를 반환해주세요.

        ## 참고 자료:
        {context}

        ## 생성된 답변:
        {generation}

        판단 (yes/no):"""
    )
    
    chain = prompt | llm | StrOutputParser()
    check_result = chain.invoke({"context": "\n\n".join(contexts), "generation": generation})

    print(f"환각 검사 결과: {check_result}")
    state["hallucination_check"] = check_result.lower()
    return state

# Edge 정의

In [82]:
# 3-1. 질문 관련성 검사 후 분기
def decide_after_relevance_check(state: GraphState):
    """
    질문 관련성 검사 결과에 따라 다음 단계를 결정합니다.
    - "yes": 질문 분해 단계로 이동
    - "no": 종료
    """
    if state["relevance_check"] == "yes":
        print("결과: 관련성 있음. 질문 분해를 진행합니다.")
        return "decompose"
    else:
        print("결과: 관련성 없음. 프로세스를 종료합니다.")
        return "end"

# 3-2. 환각 검사 후 분기
def decide_after_hallucination_check(state: GraphState):
    """
    환각 검사 결과에 따라 다음 단계를 결정합니다.
    - "yes": 환각 없음, 종료
    - "no": 환각 존재, 답변 재생성
    """
    if state["hallucination_check"] == "yes":
        print("결과: 환각 없음. 최종 답변을 반환합니다.")
        # print(state)
        return "end"
    else:
        print("결과: 환각 존재. 답변을 다시 생성합니다.")
        return "regenerate"

# 그래프 통합

## Naive RAG vs Graph RAG

In [83]:
from langgraph.graph import END, StateGraph

# 그래프 빌더 생성
def create_graph_rag():
    graph_workflow = StateGraph(GraphState)

    # 노드 추가
    graph_workflow.add_node("relevance_check", relevance_check_node)
    graph_workflow.add_node("decompose_query", decompose_query_node)
    graph_workflow.add_node("retrieve", retrieve_node)
    graph_workflow.add_node("generate", generate_node)
    graph_workflow.add_node("hallucination_check", hallucination_check_node)

    # 엣지 연결
    graph_workflow.set_entry_point("relevance_check")
    graph_workflow.add_conditional_edges(
        "relevance_check",
        decide_after_relevance_check,
        {"decompose": "decompose_query", "end": END}
    )
    graph_workflow.add_edge("decompose_query", "retrieve")
    graph_workflow.add_edge("retrieve", "generate")
    graph_workflow.add_edge("generate", "hallucination_check")
    graph_workflow.add_conditional_edges(
        "hallucination_check",
        decide_after_hallucination_check,
        {"regenerate": "generate", "end": END}
    )

    # 그래프 컴파일
    return graph_workflow.compile()

def create_meta_graph_rag():
    graph_workflow = StateGraph(GraphState)

    # 노드 추가
    graph_workflow.add_node("relevance_check", relevance_check_node)
    graph_workflow.add_node("decompose_query", decompose_query_node)
    graph_workflow.add_node("retrieve", meta_retrieve_node)
    graph_workflow.add_node("generate", generate_node)
    graph_workflow.add_node("hallucination_check", hallucination_check_node)

    # 엣지 연결
    graph_workflow.set_entry_point("relevance_check")
    graph_workflow.add_conditional_edges(
        "relevance_check",
        decide_after_relevance_check,
        {"decompose": "decompose_query", "end": END}
    )
    graph_workflow.add_edge("decompose_query", "retrieve")
    graph_workflow.add_edge("retrieve", "generate")
    graph_workflow.add_edge("generate", "hallucination_check")
    graph_workflow.add_conditional_edges(
        "hallucination_check",
        decide_after_hallucination_check,
        {"regenerate": "generate", "end": END}
    )

    # 그래프 컴파일
    return graph_workflow.compile()

def create_naive_rag():
    naive_workflow = StateGraph(GraphState)
    # 노드 추가
    naive_workflow.add_node("decompose_query", decompose_query_node)
    naive_workflow.add_node("retrieve", retrieve_node)
    naive_workflow.add_node("generate", generate_node)

    # 엣지 연결
    naive_workflow.set_entry_point("decompose_query")
    naive_workflow.add_edge("decompose_query", "retrieve")
    naive_workflow.add_edge("retrieve", "generate")
    naive_workflow.add_edge("generate", END)
    # 그래프 컴파일
    return naive_workflow.compile()

# test 정의

In [84]:
# app 생성
graph_app = create_graph_rag()
naive_app = create_naive_rag()
graph_meta_app = create_meta_graph_rag()

# HTP 그림 검사 예시 질문
inputs = {
    "original_question": "집에 창문은 2개 존재하고 크기는 적절함. 문은 집의 크기에 비해 작으며 문과 바깥이 길로 이어져 있지 않음.",
    "category": "집",
}

interupt_inputs = {
    "original_question": "오늘의 날씨는? 대한민국의 수도는? 최근 야구 경기 결과는?",
    "category": "집"
}

# Naive RAG time check

In [60]:
# 시간 초기화
execution_times = {}
# 노드수행
config = {
    "configurable": {
        "faiss_store": faiss_store, # 초기화 단계에서 생성한 faiss_store 객체
        "bm25_retriever_cache": bm25_retriever_cache # 초기화 단계에서 생성한 bm25_retriever_cache 객체
    }
}
naive_app.invoke(interupt_inputs, config=config)

# --- 최종 결과 분석 ---
print("--- Individual Node Execution Times ---")
for node_name, duration in execution_times.items():
    print(f"- {node_name}: {duration:.4f} seconds")

total_time = sum(execution_times.values())
print("\n-----------------------------------------")
print(f"Total execution time (sum of nodes): {total_time:.4f} seconds")

--- 2. 질문 분해 시작 ---
분해된 질문: ['오늘의 날씨에 대한 질문이 있다.', '대한민국의 수도에 대한 질문이 있다.', '최근 야구 경기 결과에 대한 질문이 있다.']
Execution time for decompose_query_node: 1.20 seconds
--- 3. 정보 검색 시작 (카테고리: 집) ---
  - 검색 쿼리: '오늘의 날씨에 대한 질문이 있다.'
  - 검색 쿼리: '대한민국의 수도에 대한 질문이 있다.'
  - 검색 쿼리: '최근 야구 경기 결과에 대한 질문이 있다.'
검색된 해석 Context 수: 11
Execution time for retrieve_node: 1.17 seconds
--- 4. 답변 생성 시작 ---
생성된 답변 일부: ## 최종 해석 보고서

본 보고서는 내담자의 HTP 그림 심리검사 결과를 토대로 심리상태를 종합적으로 분석한 내용입니다. 상담사가 관찰한 그림 특징을  해석 참고자료에 기반하여 논리적으로 연결하여 설명하겠습니다.

### 1. 그림 크기
내담자가 그림을 지나치게 작게 그렸다면, 이는 내면에 열등감이나 부적절감을 느끼고 있다는 가능성을 제시합니다. 이는 자...
Execution time for generate_node: 21.84 seconds
--- Individual Node Execution Times ---
- decompose_query_node: 1.1987 seconds
- retrieve_node: 1.1719 seconds
- generate_node: 21.8375 seconds

-----------------------------------------
Total execution time (sum of nodes): 24.2081 seconds


# Graph RAG time check

In [86]:
# 시간 초기화
execution_times = {}
# 노드수행
config = {
    "configurable": {
        "faiss_store": faiss_store, # 초기화 단계에서 생성한 faiss_store 객체
        "bm25_retriever_cache": bm25_retriever_cache # 초기화 단계에서 생성한 bm25_retriever_cache 객체
    }
}
graph_result = graph_app.invoke(inputs, config=config)

# --- 최종 결과 분석 ---
print("--- Individual Node Execution Times ---")
for node_name, duration in execution_times.items():
    print(f"- {node_name}: {duration:.4f} seconds")

total_time = sum(execution_times.values())
print("\n-----------------------------------------")
print(f"Total execution time (sum of nodes): {total_time:.4f} seconds")

--- 1. 질문 관련성 검사 시작 ---
질문 관련성: yes
Execution time for relevance_check_node: 0.62 seconds
결과: 관련성 있음. 질문 분해를 진행합니다.
--- 2. 질문 분해 시작 ---
분해된 질문: ['집 창문의 개수는 2개이고 크기는 적절하다.', '집 문의 크기가 집 전체에 비해 작다.', '집과 외부를 잇는 길이 그려져 있지 않다.']
Execution time for decompose_query_node: 1.37 seconds
--- 3. 정보 검색 시작 ---
  - 검색 쿼리: '집 창문의 개수는 2개이고 크기는 적절하다.'
  - 검색 쿼리: '집 문의 크기가 집 전체에 비해 작다.'
  - 검색 쿼리: '집과 외부를 잇는 길이 그려져 있지 않다.'
검색된 해석 Context 수: 12
Execution time for retrieve_node: 1.28 seconds
--- 4. 답변 생성 시작 ---
생성된 답변 일부: ### 최종 해석 보고서

이번 그림 심리검사의 분석 결과, 내담자의 심리상태를 다음과 같이 해석할 수 있습니다.

**1. 집의 구조 (창문과 문):**
내담자의 집에는 2개의 적절한 크기의 창문이 존재하지만, 문은 집의 크기에 비해 작고 바깥과 연결되는 길이 없습니다. 이 구조는 내담자가 중요한 대인관계와 외부 세상과의 연결에 어려움을 느끼고 있음을 나...
Execution time for generate_node: 17.86 seconds
--- 5. 환각 검사 시작 ---
환각 검사 결과: yes
Execution time for hallucination_check_node: 0.74 seconds
결과: 환각 없음. 최종 답변을 반환합니다.
--- Individual Node Execution Times ---
- relevance_check_node: 0.6187 seconds
- decompose_query_node: 1.3717 seconds
- ret

# Meta Graph RAG time check

In [90]:
# 시간 초기화
execution_times = {}
config = {
    "configurable": {
        "faiss_store": faiss_store, # 초기화 단계에서 생성한 faiss_store 객체
        "bm25_retriever_cache": bm25_retriever_cache # 초기화 단계에서 생성한 bm25_retriever_cache 객체
    }
}
# 노드수행

meta_graph_result = graph_app.invoke(inputs, config=config)

# --- 최종 결과 분석 ---
print("--- Individual Node Execution Times ---")
for node_name, duration in execution_times.items():
    print(f"- {node_name}: {duration:.4f} seconds")

total_time = sum(execution_times.values())
print("\n-----------------------------------------")
print(f"Total execution time (sum of nodes): {total_time:.4f} seconds")

--- 1. 질문 관련성 검사 시작 ---
질문 관련성: yes
Execution time for relevance_check_node: 0.57 seconds
결과: 관련성 있음. 질문 분해를 진행합니다.
--- 2. 질문 분해 시작 ---
분해된 질문: ['집 창문의 개수는 2개이고 크기는 적절하다.', '집 문의 크기가 집 전체에 비해 작다.', '문과 외부를 잇는 길이 그려져 있지 않다.']
Execution time for decompose_query_node: 1.48 seconds
--- 3. 정보 검색 시작 ---
  - 검색 쿼리: '집 창문의 개수는 2개이고 크기는 적절하다.'
  - 검색 쿼리: '집 문의 크기가 집 전체에 비해 작다.'
  - 검색 쿼리: '문과 외부를 잇는 길이 그려져 있지 않다.'
검색된 해석 Context 수: 13
Execution time for retrieve_node: 1.80 seconds
--- 4. 답변 생성 시작 ---
생성된 답변 일부: ## 최종 해석 보고서

내담자가 그린 집 그림을 분석한 결과에 따르면, 집의 구조와 요소들이 내담자의 심리적 상태와 대인 관계의 특성을 반영하고 있습니다.

1. **창문의 수와 크기**: 내담자가 그린 창문은 2개로, 크기는 적절합니다. 이는 그가 현재 대인관계에서 적절한 수준의 개방성을 유지하고 있다는 것을 의미합니다. 그러나 대인관계에 대한 적극적인...
Execution time for generate_node: 12.73 seconds
--- 5. 환각 검사 시작 ---
환각 검사 결과: yes
Execution time for hallucination_check_node: 0.72 seconds
결과: 환각 없음. 최종 답변을 반환합니다.
--- Individual Node Execution Times ---
- relevance_check_node: 0.5671 seconds
- decompose_query_node: 1.4779 seconds
- ret

# LLM as a judge
평가자 gpt-4o
[https://galtea.ai/2025/05/02/evaluation-of-judges.html]
